In [ ]:
! pip install torch torchvision

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.1 MB 1.1 MB/s eta 0:03:02
   ---------------------------------------- 0.8/204.1 MB 1.1 MB/s eta 0:03:08
   ---------------------------------------- 0.8/204.1 MB 1.1 MB/s eta 0:03:08
   ---------------------------------------- 1.0/204.1 MB 853.0 kB/s eta 0:03:59
   ---------------------------------------- 1.0/204.1 MB 853.0 kB/s eta 0:03:59
   ---------------------------------------- 1.0/204.1 MB 853.0 kB/s eta 0:03:59
   ---------------------------------------- 1.0/204.1 MB 853.0 kB/s eta 0:03:59
   ---------------------------------------- 1.3/204.1 MB 658.2 kB/s eta 0:05:09
   ---------------------------------------- 1.3/204.1 MB 658.2 kB/s eta 0:05:09
   ---------------------------------------- 1.3/204.1 MB 658.2 kB/s eta 0

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\ssl.py", line 1252, in r

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [13]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='my_dir/logs')

In [3]:
df = pd.read_csv('/ParisHousing.csv')

In [4]:
X = df.drop(columns = ['price']).values
y = df['price'].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=3)

In [6]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [7]:
class MobileDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# Создаём Dataset
train_dataset = MobileDataset(X_train_tensor, y_train_tensor)
test_dataset = MobileDataset(X_test_tensor, y_test_tensor)

In [8]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [9]:
# Определение модели
class NeuralNet(nn.Module):
    def __init__(self, input_size=X_train_tensor.shape[1],
                 hidden_size=64, num_classes=1):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)

        return out

In [10]:
model = NeuralNet()

In [11]:
# Параметры обучения
num_epochs = 100
learning_rate = 0.01

In [21]:
y_pred = model(X_train_tensor)
y_train_tensor.unsqueeze(1).shape

torch.Size([8000, 1])

In [22]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

def train_model(iter):
    for epoch in range(iter):
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor.unsqueeze(1))
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_model(10)
writer.flush()

In [23]:
writer.close()

In [24]:
correct = 0
total = 0

model.eval()
predictions = []
true_values = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)

        # Сохраняем предсказания и истинные значения
        predictions.extend(y_pred.numpy())  # Предсказания модели
        true_values.extend(y_batch.numpy())  # Истинные значения

mse = mean_absolute_error(true_values, predictions)
print(f"Среднеквадратичная ошибка (MSE): {mse:.4f}")

Среднеквадратичная ошибка (MSE): 6982715254547553452032.0000
